In [1]:
import collections
import json
import uuid

In [2]:
with open("pikov.json") as f:
    core = json.load(f)

guid_map = core["guidMap"]

In [3]:
#json.dumps(core, indent=2, sort_keys=True)

## Build names mapping

To make it a little easier to check that I'm using the correct guids, construct a mapping from names back to `guid`.

Note: this adds a constraint that no two nodes have the same name, which should not be enforced for general semantic graphs.

In [4]:
names = {}
for key, edges in guid_map.items():
    for edge, value in edges.items():
        if edge == "169a81aefca74e92b45e3fa03c7021df":
            if value["string"] in names:
                raise ValueError('name: "{}" defined twice'.format(value["string"]))
            names[value["string"]] = key
     
# names

## Semantic Graph Helper Classes

We want to wrap our semantic graph with Python classes. This allows us to interact with Python objects to modify the `guid_map`.

In [5]:
class SemanticGraphNode(object):
    def __init__(self, ctor, guid_map, guid=None):
        if guid is None:
            guid = uuid.uuid4().hex
        self._guid = guid
        self._guid_map = guid_map

        if guid in self._guid_map:
            self._edges = self._guid_map[guid]
        else:
            self._edges = {
                names["ctor"]: {"guid": ctor},
            }
            self._guid_map[guid] = self._edges
    
    def __repr__(self):
        return '"{}": {}'.format(
            self._guid,
            json.dumps(self._edges, indent=2, sort_keys=True))


class AbstractSemanticGraphProperty(object):
    def __init__(self, label_guid):
        self._label_guid = label_guid

    def __get__(self, obj, type=None):
        return self.from_json(obj, obj._edges.get(self._label_guid, {}))

    def __set__(self, obj, value):
        obj._edges[self._label_guid] = self.to_json(value)

### Semantic Graph Properties

These classes encode the core types used in the semantic graph. When classes use these properties, the `guid_map` is updated with the correct serialization of the property.

In [6]:
class GuidProperty(AbstractSemanticGraphProperty):
    def __init__(self, label_guid, cls):
        super().__init__(label_guid)
        self._cls = cls

    def from_json(self, obj, value):
        guid = value.get("guid")
        if guid is None:
            return None
        return self._cls(obj._guid_map, guid=guid)
    
    def to_json(self, value):
        return {"guid": value._guid}

    
def make_guid_property(wrapped):
    def __init__(self, label_guid):
        GuidProperty.__init__(self, label_guid, wrapped)

    return type(
        wrapped.__name__ + "Property",
        (GuidProperty,),
        {
            "__init__": __init__,
        }
    )

        
class Int64Property(AbstractSemanticGraphProperty):
    def from_json(self, obj, value):
        out = value.get("int64")
        if out is None:
            return None
        return int(out)

    def to_json(self, value):
        return {"int64": str(value)}
            
        
class StringProperty(AbstractSemanticGraphProperty):
    def from_json(self, obj, value):
        return value.get("string")

    def to_json(self, value):
        return {"string": value}

## Pikov Classes

These classes are the core resources used in defining a "Pikov" file.

Note: ideally these classes could be derived from the graph itself, but I don't (yet) encode type or field information in the `pikov.json` semantic graph.

In [7]:
class Resource(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["resource"], guid_map, guid=guid)
    
    relative_path = StringProperty(names["resource.relative_path"])


ResourceProperty = make_guid_property(Resource)

In [8]:
class Coordinate(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["coordinate"], guid_map, guid=guid)
    
    x = Int64Property(names["coordinate.x"])
    y = Int64Property(names["coordinate.y"])


CoordinateProperty = make_guid_property(Coordinate)

In [9]:
class Vector(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["vector"], guid_map, guid=guid)
    
    x = Int64Property(names["vector.x"])
    y = Int64Property(names["vector.y"])


VectorProperty = make_guid_property(Vector)

In [10]:
class Rectangle(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["rectangle"], guid_map, guid=guid)
    
    anchor = CoordinateProperty(names["rectangle.anchor"])
    width = Int64Property(names["rectangle.width"])
    height = Int64Property(names["rectangle.height"])


RectangleProperty = make_guid_property(Rectangle)

In [11]:
class Bitmap(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["bitmap"], guid_map, guid=guid)
    
    resource = ResourceProperty(names["bitmap.resource"])
    rectangle = RectangleProperty(names["bitmap.rectangle"])
    
    
BitmapProperty = make_guid_property(Bitmap)

In [12]:
class Frame(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["frame"], guid_map, guid=guid)
    
    duration_microseconds = Int64Property(names["frame.duration_microseconds"])
    bitmap = ResourceProperty(names["frame.bitmap"])


FrameProperty = make_guid_property(Frame)

In [13]:
class Sprite(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["sprite"], guid_map, guid=guid)
    
    frame = FrameProperty(names["sprite.frame"])
    position = CoordinateProperty(names["sprite.position"])
    
SpriteProperty = make_guid_property(Sprite)

## Gamekitty

Create instances of the Pikov classes to define a concrete Pikov graph, based on my "gamekitty" animations.

In [14]:
resource = Resource(guid_map, guid="650eeac69efb4d9096964e3c3fb4e13e")
resource.relative_path = "./gamekitty.png"
resource

"650eeac69efb4d9096964e3c3fb4e13e": {
  "4e09f9df1fdf4eb4964ff9ed1b375dbb": {
    "string": "./gamekitty.png"
  },
  "aba6ac79fd3d409da860a77c90942852": {
    "guid": "6ecf1345ea0b4865b92569971b100b09"
  }
}

In [15]:
origin = Coordinate(guid_map, "1c3e3474bf5b456d9bc1e9591f126167")
origin.x = 0
origin.y = 0
origin

"1c3e3474bf5b456d9bc1e9591f126167": {
  "66b25e276cfb4d83a7032baaa4369b6f": {
    "int64": "0"
  },
  "825142afc2934fbcb5126e149ac5ba31": {
    "int64": "0"
  },
  "aba6ac79fd3d409da860a77c90942852": {
    "guid": "bfa8113cb5e5436ebd76ab5418b7efd1"
  }
}

In [16]:
sprite = Sprite(guid_map, guid="4148508493084e4f9e51e8c3e75cea9a")
core["root"] = "4148508493084e4f9e51e8c3e75cea9a"
sprite.position = origin
sprite

"4148508493084e4f9e51e8c3e75cea9a": {
  "2175a23088d347cb9256b7f6c6eae310": {
    "guid": "1c3e3474bf5b456d9bc1e9591f126167"
  },
  "aba6ac79fd3d409da860a77c90942852": {
    "guid": "158aafa594b44474a7da66a8cfa419f0"
  }
}